In [1]:
import polars as pl

In [7]:
clubs_df = pl.read_csv('data/After School Club.csv')
events_df = pl.read_csv('data/Events.csv', try_parse_dates=True)
display(
    clubs_df, events_df
)

Day of Week,After School Club
str,str
"""Monday""","""Art Club"""
"""Tuesday""","""Karate Club"""
"""Wednesday""","""Homework Club"""
"""Thursday""","""Coding Club"""
"""Friday""","""Gymnastics Clu…"


Date,Event
date,str
2023-04-17,"""First Day of T…"
2023-04-20,"""Paint-a-Pot Tr…"
2023-04-24,"""Inset Day"""
2023-04-25,"""Natural Histro…"
2023-05-03,"""World Book Day…"
2023-05-05,"""Non-uniform Da…"
2023-05-16,"""Inset Day"""
2023-05-24,"""Stongehenge Tr…"
2023-05-26,"""Last Day of Te…"


In [23]:
weekdays_mapping = {1: 'Monday', 2: 'Tuesday', 3: 'Wednesday', 4: 'Thursday', 5: 'Friday'}

(
    events_df
    # .set_sorted('Date')
    .sort('Date')
    .upsample(
        time_column='Date', every='1d', maintain_order=True
    )
    .with_columns(
        pl.col('Date')
        .dt.weekday()
        .map_dict(weekdays_mapping)
        .alias('Day of Week'),
        pl.col('Event').fill_null('N/A')
    )
    .filter( 
        pl.col('Day of Week').is_not_null() 
    )
    .join(
        clubs_df, on='Day of Week', how='left'
    )
)


Date,Event,Day of Week,After School Club
date,str,str,str
2023-04-17,"""First Day of T…","""Monday""","""Art Club"""
2023-04-18,"""N/A""","""Tuesday""","""Karate Club"""
2023-04-19,"""N/A""","""Wednesday""","""Homework Club"""
2023-04-20,"""Paint-a-Pot Tr…","""Thursday""","""Coding Club"""
2023-04-21,"""N/A""","""Friday""","""Gymnastics Clu…"
2023-04-24,"""Inset Day""","""Monday""","""Art Club"""
2023-04-25,"""Natural Histro…","""Tuesday""","""Karate Club"""
2023-04-26,"""N/A""","""Wednesday""","""Homework Club"""
2023-04-27,"""N/A""","""Thursday""","""Coding Club"""
